In [1]:
!pip install datasets

In [2]:
!pip install sacrebleu

In [6]:
!pip install lexrank

In [5]:
!pip install tqdm

In [7]:
from summa import summarizer
from datasets import load_dataset

cnndaily = load_dataset('cnn_dailymail', '3.0.0')
billsum = load_dataset('billsum')
reddit = load_dataset('reddit_tifu', 'short')

/home/apatel/miniconda3/envs/summariser/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset cnn_dailymail (/home/apatel/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 142.04it/s]
Using custom data configuration default
Reusing dataset billsum (/home/apatel/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959)
100%|███████████████████████████████████████████| 3/3 [00:00<00:00, 1207.92it/s]
Reusing dataset reddit_tifu (/home/apatel/.cache/huggingface/datasets/reddit_tifu/short/1.1.0/1c73fb08807b54ec26b025829b2a3d90c6f7466dac20801c825571af9514c049)
100%|██████████████████████████████████████████

In [14]:
reddit_l = load_dataset('reddit_tifu', 'long')

Dataset reddit_tifu downloaded and prepared to /home/apatel/.cache/huggingface/datasets/reddit_tifu/long/1.1.0/1c73fb08807b54ec26b025829b2a3d90c6f7466dac20801c825571af9514c049. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 557.23it/s]


### CNN Daily Mail results

In [9]:
from tqdm import tqdm
from datetime import datetime
references = []
predictions = []
start = datetime.now()
for instance in tqdm(cnndaily['test']):
    query = instance['article']
    reference = instance['highlights']
    prediction = summarizer.summarize(query)
    references.append([[reference]])
    predictions.append([prediction])
end = datetime.now()

100%|█████████████████████████████████████| 11490/11490 [01:55<00:00, 99.07it/s]


In [10]:
print(end-start)

0:01:55.983729


In [133]:
references_tokens = [[[r]] for r in references]
predictions_tokens = [[p] for p in predictions]

In [57]:
reference_tokens = []
prediction_tokens = []
for i in range(len(references)):
    reference_tokens.append([references[i].split(' ')])
    prediction_tokens.append([predictions[i].split(' ')])

In [35]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
tokenizer = Tokenizer(nlp.vocab)
text = predictions[0]
tokens = tokenizer(text)

In [11]:
from datasets import load_metric, list_metrics
print(list_metrics())
metric = load_metric('sacrebleu', 'cnn_dailymail')
# metric.compute(predictions=[["I am performing team analysis"]], references=[[["I am performing team analysis"]]])
# metric.compute(predictions=[["ant"], ["ball bat cat"], ["vase van ball"]], references=[[["ant"]],[["ball bat cat"]],[["vase van ball"]]])
metric.compute(predictions=predictions, references=references)

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'chrf', 'code_eval', 'comet', 'competition_math', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'pearsonr', 'perplexity', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'wer', 'wiki_split', 'xnli', 'xtreme_s']


{'score': 6.309932009942896,
 'counts': [441472, 194960, 97678, 57874],
 'totals': [2371005, 2359515, 2348025, 2336535],
 'precisions': [18.61961488904494,
  8.262715007109511,
  4.160006814237497,
  2.4769156036609767],
 'bp': 1.0,
 'sys_len': 2371005,
 'ref_len': 737699}

In [69]:
from datasets import load_metric, list_metrics
metric = load_metric('bertscore', 'cnn_dailymail')
metric.compute(predictions=predictions, references=references, lang="en")

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'chrf', 'code_eval', 'comet', 'competition_math', 'coval', 'cuad', 'exact_match', 'f1', 'frugalscore', 'glue', 'google_bleu', 'indic_glue', 'mae', 'mahalanobis', 'matthews_correlation', 'mauve', 'mean_iou', 'meteor', 'mse', 'pearsonr', 'perplexity', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'ter', 'wer', 'wiki_split', 'xnli', 'xtreme_s']


KeyboardInterrupt: 

In [13]:
metric = load_metric('rouge')
# metric.compute(predictions=[["a"], ["b"], ["v"]], references=[[['a']],[['b']],[['v']]])
metric.compute(predictions=predictions, references=references)

{'rouge1': AggregateScore(low=Score(precision=0.20304941492837966, recall=0.5470768318858245, fmeasure=0.2727423973276021), mid=Score(precision=0.20522470515904606, recall=0.549896025884278, fmeasure=0.2746197785157393), high=Score(precision=0.2074230545767761, recall=0.5526244909624248, fmeasure=0.2764987309714956)),
 'rouge2': AggregateScore(low=Score(precision=0.0852720945638325, recall=0.22536057942619028, fmeasure=0.11355126719286165), mid=Score(precision=0.08669817659566426, recall=0.2279327398587372, fmeasure=0.11502039415595744), high=Score(precision=0.08802862789426713, recall=0.23040067596756503, fmeasure=0.11644403848059859)),
 'rougeL': AggregateScore(low=Score(precision=0.1345304087950258, recall=0.36088120910699656, fmeasure=0.17968503707078967), mid=Score(precision=0.13612183306950498, recall=0.36331944867664023, fmeasure=0.18125464408721909), high=Score(precision=0.13765045827866612, recall=0.36566623081908267, fmeasure=0.18270020187384348)),
 'rougeLsum': AggregateScor

## Lead Summarizer

In [41]:
from nltk.tokenize import word_tokenize, sent_tokenize
def lead(dataset, length):
    predictions = []
    references = []
    
    for instance in tqdm(dataset['train']):
        sentences = sent_tokenize(instance['documents'])
        
    # summarize and keep track of summary word count
        summary = ""
        sentence_index, summary_word_count = 0, 0

        while summary_word_count <= length and sentence_index < len(sentences):
            sentence = sentences[sentence_index]
            summary += sentence + " "
            sentence_index += 1
            summary_word_count += len(word_tokenize(sentence))
        predictions.append([summary.strip()])
        references.append([[instance['tldr']]])
    return predictions, references

In [31]:
def metrics(metric, predictions, references):
    metric = load_metric(metric)
    print(metric.compute(predictions=predictions, references=references))

In [27]:
p, r = lead(cnndaily, 51)

100%|████████████████████████████████████| 11490/11490 [00:13<00:00, 860.79it/s]


In [32]:
metrics('rouge', p, r)

{'rouge1': AggregateScore(low=Score(precision=0.3400507411364121, recall=0.4135885467056839, fmeasure=0.361234896239558), mid=Score(precision=0.34230609203393925, recall=0.4162487385505991, fmeasure=0.36330840024114824), high=Score(precision=0.34468493044041304, recall=0.41880820794454054, fmeasure=0.3653851548732092)), 'rouge2': AggregateScore(low=Score(precision=0.1422291358149959, recall=0.1740579493361313, fmeasure=0.15126558460150766), mid=Score(precision=0.14418077470988894, recall=0.1764227451949611, fmeasure=0.15312652358368323), high=Score(precision=0.14616277136258826, recall=0.1788546979798391, fmeasure=0.15510614804864395)), 'rougeL': AggregateScore(low=Score(precision=0.21376032223790262, recall=0.2647797409833294, fmeasure=0.2288824094264809), mid=Score(precision=0.21559237062242012, recall=0.26697030394002563, fmeasure=0.23064026951070044), high=Score(precision=0.21732391566602807, recall=0.26932865363811725, fmeasure=0.23231099248528023)), 'rougeLsum': AggregateScore(lo

In [35]:
p, r = lead(billsum, 179)

100%|██████████████████████████████████████| 3269/3269 [00:08<00:00, 388.57it/s]


In [36]:
metrics('sacrebleu', p, r)

{'score': 9.303562244551324, 'counts': [284562, 106387, 56703, 32283], 'totals': [932300, 929031, 925762, 922493], 'precisions': [30.522578569130108, 11.451393979318237, 6.125008371482087, 3.499538749887533], 'bp': 1.0, 'sys_len': 932300, 'ref_len': 693288}


In [37]:
metrics('rouge', p, r)

{'rouge1': AggregateScore(low=Score(precision=0.35678737497301916, recall=0.4611507690508963, fmeasure=0.3617973278497395), mid=Score(precision=0.36212056800587106, recall=0.46708819173245725, fmeasure=0.36557816577612784), high=Score(precision=0.3679165601731733, recall=0.47332139835452497, fmeasure=0.3695181192435316)), 'rouge2': AggregateScore(low=Score(precision=0.14453504922988075, recall=0.19460248433791758, fmeasure=0.14801563328860443), mid=Score(precision=0.14809491949842363, recall=0.19910578445898047, fmeasure=0.15107080861484976), high=Score(precision=0.1514268628841941, recall=0.2038645466230141, fmeasure=0.15408240124725953)), 'rougeL': AggregateScore(low=Score(precision=0.21979995815647718, recall=0.29927745186165317, fmeasure=0.226508601195775), mid=Score(precision=0.2231665737278232, recall=0.3040965371718626, fmeasure=0.2293135935147173), high=Score(precision=0.22646114003391404, recall=0.3091233420140177, fmeasure=0.23225191558151045)), 'rougeLsum': AggregateScore(lo

In [42]:
p, r = lead(reddit_l, 21)

100%|███████████████████████████████████| 42139/42139 [00:33<00:00, 1239.48it/s]


In [43]:
metrics('sacrebleu', p, r)

{'score': 2.107068134414731, 'counts': [337696, 67850, 23724, 2538], 'totals': [1690414, 1648275, 1606136, 1563997], 'precisions': [19.977118031440817, 4.116424747084073, 1.4770853775769923, 0.16227652610586848], 'bp': 1.0, 'sys_len': 1690414, 'ref_len': 1115133}


In [44]:
metrics('rouge', p, r)

{'rouge1': AggregateScore(low=Score(precision=0.1290154604088615, recall=0.20295169788895498, fmeasure=0.14459886445156672), mid=Score(precision=0.12988456278688804, recall=0.20416864043513872, fmeasure=0.14543274284896096), high=Score(precision=0.13074385929478322, recall=0.205424361433069, fmeasure=0.14622529810218984)), 'rouge2': AggregateScore(low=Score(precision=0.01698663989880662, recall=0.027694217222659006, fmeasure=0.019068816717172817), mid=Score(precision=0.017349972646377332, recall=0.028271383013463616, fmeasure=0.019453276246343954), high=Score(precision=0.017706847678193367, recall=0.028832242183375125, fmeasure=0.019834814899203003)), 'rougeL': AggregateScore(low=Score(precision=0.0931690427997301, recall=0.1519603280779647, fmeasure=0.1056582703515423), mid=Score(precision=0.09380437409886067, recall=0.15292040870004453, fmeasure=0.10625315542341346), high=Score(precision=0.09438538280097448, recall=0.15397021345674108, fmeasure=0.1068594219773597)), 'rougeLsum': Aggr

## Billsum Dataset

In [16]:
from tqdm import tqdm
bs_references = []
bs_predictions = []
for instance in tqdm(billsum['test']):
    query = instance['text']
    reference = instance['summary']
    prediction = summarizer.summarize(query)
    bs_references.append([[reference]])
    bs_predictions.append([prediction])

100%|███████████████████████████████████████| 3269/3269 [06:00<00:00,  9.06it/s]


In [20]:
from datasets import load_metric, list_metrics
# print(list_metrics())
metric = load_metric('sacrebleu', 'billsum')
metric.compute(predictions=bs_predictions, references=bs_references)

{'score': 8.105573611534249,
 'counts': [347492, 139619, 74220, 43066],
 'totals': [1381650, 1378381, 1375112, 1371843],
 'precisions': [25.150508450041617,
  10.129202303281893,
  5.397378540802495,
  3.1392805153359387],
 'bp': 1.0,
 'sys_len': 1381650,
 'ref_len': 693288}

In [22]:
metric = load_metric('rouge', 'billsum')
metric.compute(predictions=bs_predictions, references=bs_references)

{'rouge1': AggregateScore(low=Score(precision=0.29366946733775157, recall=0.5112937396967577, fmeasure=0.3428276667587242), mid=Score(precision=0.29877202660420865, recall=0.5157646991919496, fmeasure=0.34676238323844577), high=Score(precision=0.30374532123829495, recall=0.5199161967116457, fmeasure=0.35060077919906835)),
 'rouge2': AggregateScore(low=Score(precision=0.12427429834323754, recall=0.21326877013405474, fmeasure=0.14384972348842248), mid=Score(precision=0.12712790886309727, recall=0.2165711503434628, fmeasure=0.1462389284759088), high=Score(precision=0.12980079475252576, recall=0.22009962568757668, fmeasure=0.1486345829933406)),
 'rougeL': AggregateScore(low=Score(precision=0.17624643817551147, recall=0.3169406359766278, fmeasure=0.2068998380791264), mid=Score(precision=0.17928482519952144, recall=0.32036665235190565, fmeasure=0.20935721845179955), high=Score(precision=0.18243061526196488, recall=0.3237740310564653, fmeasure=0.2117242841424929)),
 'rougeLsum': AggregateScor

### Reddit Tifu long

In [168]:
# for i in reddit['train']:
#     print(i)
reddit_l['train']
# reddit_samples

Dataset({
    features: ['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title'],
    num_rows: 79740
})

In [ ]:
from tqdm import tqdm
rt_references = []
rt_predictions = []
for instance in tqdm(reddit_l['train']):
    query = instance['documents']
    reference = instance['tldr']
    prediction = summarizer.summarize(query)
    rt_references.append([[reference]])
    rt_predictions.append([prediction])

 78%|████████████████████████████▏       | 33000/42139 [02:59<00:53, 170.07it/s]

In [171]:
metric = load_metric('sacrebleu', 'reddit_tifu')
metric.compute(predictions=rt_predictions, references=rt_references)

{'score': 1.0934596613112226,
 'counts': [631636, 137593, 46778, 10438],
 'totals': [7501362, 7421622, 7341882, 7262142],
 'precisions': [8.420284209720847,
  1.8539478297331768,
  0.6371390877706833,
  0.1437316978929908],
 'bp': 1.0,
 'sys_len': 7501362,
 'ref_len': 1227936}

In [182]:
metric = load_metric('rouge', 'reddit_tifu')
metric.compute(predictions=rt_predictions, references=rt_references)

{'rouge1': AggregateScore(low=Score(precision=0.05523798696925347, recall=0.20401740340951258, fmeasure=0.07890371367443742), mid=Score(precision=0.05575873675325173, recall=0.20550285954471845, fmeasure=0.07955652864490603), high=Score(precision=0.056299808237724465, recall=0.20708013464007252, fmeasure=0.08021896320345584)),
 'rouge2': AggregateScore(low=Score(precision=0.011364385652004866, recall=0.04279479796691269, fmeasure=0.016226901198465468), mid=Score(precision=0.011544961509948444, recall=0.043348103911979505, fmeasure=0.016444098951665286), high=Score(precision=0.011737295961418669, recall=0.043930011353607044, fmeasure=0.016688435698417936)),
 'rougeL': AggregateScore(low=Score(precision=0.038971708600889374, recall=0.14724522135972978, fmeasure=0.055704606725571364), mid=Score(precision=0.039360320202898445, recall=0.14841679282874598, fmeasure=0.056175743461478765), high=Score(precision=0.03976735124406718, recall=0.14967170485257703, fmeasure=0.056693035839949925)),
 '

In [94]:
print(metric)

Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_length': reference_length
Examples

## Lexrank

In [45]:
from lexrank import STOPWORDS, LexRank
from tqdm import tqdm
import nltk
nltk.download('punkt')
# word_counts, doc, summary, summary time

[nltk_data] Downloading package punkt to /home/apatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
# document level median length
def lexrank(dataset, length):
    start = datetime.now()
    lxr = LexRank(text, stopwords=STOPWORDS['en'])
    # math.ceil(cnndataset)/median sentence length
    
    from nltk import tokenize
    references = []
    predictions = []
    for instance in tqdm(dataset['test']):
        query = instance['article']
        reference = instance['highlights']
        sentences = tokenize.sent_tokenize(query)
        prediction = lxr.get_summary(sentences, summary_size=5, threshold=None)
        references.append([[reference]])
        predictions.append([prediction])
    end = datetime.now()
    return 

In [ ]:
p, r = lexrank(cnndaily, 

In [ ]:
metrics('sacrebleu', p, r)

In [16]:
from datasets import load_metric
metric = load_metric('sacrebleu', 'cnn_dailymail')
metric.compute(predictions=cnnpredictions_t, references=cnnreferences_t)

{'score': 7.071632497961682,
 'counts': [383173, 145379, 78918, 47834],
 'totals': [1720158, 1708668, 1697178, 1685688],
 'precisions': [22.275453766456337,
  8.508323442588027,
  4.649954218119726,
  2.837654417662106],
 'bp': 1.0,
 'sys_len': 1720158,
 'ref_len': 737699}

In [18]:
metric = load_metric('rouge', 'cnn_dailymail')
metric.compute(predictions=cnnpredictions_t, references=cnnreferences_t)

{'rouge1': AggregateScore(low=Score(precision=0.2149415635760081, recall=0.5021878634020653, fmeasure=0.29097057246558644), mid=Score(precision=0.21662229788452544, recall=0.5049621296342061, fmeasure=0.2927360223608766), high=Score(precision=0.2181612414574238, recall=0.5077388949018342, fmeasure=0.2943686541346999)),
 'rouge2': AggregateScore(low=Score(precision=0.0874714252645371, recall=0.20820342979035544, fmeasure=0.1191443505333403), mid=Score(precision=0.0886568907883392, recall=0.21095542142136708, fmeasure=0.12061179814482456), high=Score(precision=0.08999349528756678, recall=0.21364179294930036, fmeasure=0.12225710350723286)),
 'rougeL': AggregateScore(low=Score(precision=0.13708786781427879, recall=0.3259230987142806, fmeasure=0.1865006127540178), mid=Score(precision=0.13818253897149207, recall=0.3282953782368314, fmeasure=0.1877721503732314), high=Score(precision=0.1393212275462485, recall=0.33081009499539116, fmeasure=0.18913822705492145)),
 'rougeLsum': AggregateScore(lo

In [223]:
text = []
for instance in tqdm(billsum['test']):
    query = instance['text']
    text.append(query)
    
lxr = LexRank(text, stopwords=STOPWORDS['en'])

100%|█████████████████████████████████████| 3269/3269 [00:00<00:00, 9482.29it/s]


ValueError: documents are not informative

In [ ]:
from nltk import tokenize
cnnreferences = []
cnnpredictions = []
for instance in tqdm(dataset['test']):
    query = instance['text']
    reference = instance['summary']
    sentences = tokenize.sent_tokenize(reference)
    print(sentences)
    prediction = lxr.get_summary(sentences, summary_size=55, threshold=None)
    cnnreferences.append(reference)
    cnnpredictions.append(prediction)